In [0]:
%run ../utils

In [0]:

display(dbutils.fs.ls("/mnt/bronze-layer/2025-05-12"))

In [0]:
dbutils.widgets.text("date_val", "")
date_val = dbutils.widgets.get("date_val")

In [0]:
date_val = calculate_date(date_val)

In [0]:
input_path = f'/mnt/bronze-layer/{date_val}/*.json'

df = spark.read.json(input_path)



In [0]:
res_df = df.select('metadata.count')
print(res_df)
initial_record_count = res_df.collect()[0]['count']
print(initial_record_count)

In [0]:
df.withColumn('feature',explode('features')).select('feature.properties').show(truncate=False)

In [0]:
# Explode the 'features' column to create a new row for each element in the array
df = df.withColumn('fearures', explode(col('features')))

# Select the 'properties' field from the exploded 'features' column
df = df.select('fearures.properties')

# Display the resulting DataFrame


In [0]:
# List of columns to be extracted from the 'properties' field
col_lst = ['alert','cdi','code','detail','dmin','felt','gap','ids','mag','magType','mmi','net','nst','place','rms','sig','sources','status','time','title','tsunami','type','types','tz','updated','url']

# Loop through each column in the list and create a new column in the DataFrame
# by extracting the corresponding field from the 'properties' column
for column in col_lst:
    df = df.withColumn(column, col(f'properties.{column}'))

# Drop the 'properties' column as it is no longer needed
df = df.drop('properties')

In [0]:
# Count the number of records in the DataFrame after transformations
final_count = df.count()

# Check if the final record count matches the initial record count
if initial_record_count != final_count:
    # Raise an error if the record counts do not match
    raise ValueError(f"Expected {initial_record_count} records, but found {final_count} records.")

In [0]:

df = df.withColumn('timestamp_column', from_unixtime(col('time')/1000)).withColumn('updated_timestamp', from_unixtime(col('updated')/1000)).withColumn('ingest_ts',current_timestamp())

df.select('time','timestamp_column','updated','updated_timestamp','ingest_ts').display()

df.write.mode('overwrite').format('parquet').save(f'/mnt/silver-layer/{date_val}/earthqukedata')

In [0]:
print(df.columns)

In [0]:
location_url = "https://earthquake.usgs.gov/realtime/product/nearby-cities/ci41152672/ci/1747237148557/nearby-cities.json"


df = df.withColumn(
    "location_url",
    concat(
        lit("https://earthquake.usgs.gov/realtime/product/nearby-cities/"),
        col("net"),
        col("code"),
        lit("/"),
        col("net"),
        lit("/"),
        col("updated"),
        lit("/nearby-cities.json"),
    )
).withColumn('row_id',concat(col('net'),col('code')))

df.display()

In [0]:
location_url_Id_lst = [{'location_url':x['location_url'],'row_id':x['row_id']} for x in df.select('location_url','row_id').collect()]
print(len(location_url_Id_lst))

In [0]:
location_url_Id_lst

In [0]:
result_set = []
cnt = 0
for data in location_url_Id_lst:
  location_url = data['location_url']
  row_id = data['row_id']
  reqest_res = requests.get(location_url)
  if reqest_res.status_code == 200:
    if cnt <=10:
      for val in reqest_res.json():
          val['row_id'] = row_id
          result_set.append(val)
    else:
      break
    cnt += 1
print(result_set)




In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema
schema = StructType([
    StructField("distance", IntegerType()),
    StructField("direction", StringType()),
    StructField("name", StringType()),
    StructField("longitude", StringType()),
    StructField("latitude", StringType()),
    StructField("row_id", StringType())
    # Add more fields as per your result_set structure
])

res_df = spark.createDataFrame(result_set,schema)

res_df.write.mode('overwrite').format('parquet').save(f'/mnt/bronze-layer/{date_val}/earthquake_nearby_cities/')

In [0]:
res_df.display()